<a href="https://colab.research.google.com/github/markoBel3/Generative-Chatbot-TransformerArchitecture/blob/main/zavrsniFOI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
!pip install contractions
import contractions
import nltk
nltk.download('punkt')
import string
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
import numpy as np
import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Dataset source: https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html

In [58]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/Cornel-Question&Answers-shorterVersion3.csv")
pd.set_option('display.max_colwidth', None)
data.head()

,question,answer
0,Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again.,"Well, I thought we'd start with pronunciation, if that's okay with you."
1,"Well, I thought we'd start with pronunciation, if that's okay with you.",Not the hacking and gagging and spitting part. Please.
2,Not the hacking and gagging and spitting part. Please.,Okay... then how 'bout we try out some French cuisine. Saturday? Night?
3,You're asking me out. That's so cute. What's your name again?,Forget it.
4,"No, no, it's my fault -- we didn't have a proper introduction ---",Cameron.


In [59]:
data['question_tokenized'] = data['question'].apply(lambda x: [contractions.fix(word) for word in str(x).split()])
data['answer_tokenized'] = data['answer'].apply(lambda x: [contractions.fix(word) for word in str(x).split()])
data.head()

,question,answer,question_tokenized,answer_tokenized
0,Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again.,"Well, I thought we'd start with pronunciation, if that's okay with you.","[Can, we, make, this, quick?, Roxanne, Korrine, and, Andrew, Barrett, are, having, an, incredibly, horrendous, public, break-, up, on, the, quad., Again.]","[Well,, I, thought, we would, start, with, pronunciation,, if, that is, okay, with, you.]"
1,"Well, I thought we'd start with pronunciation, if that's okay with you.",Not the hacking and gagging and spitting part. Please.,"[Well,, I, thought, we would, start, with, pronunciation,, if, that is, okay, with, you.]","[Not, the, hacking, and, gagging, and, spitting, part., Please.]"
2,Not the hacking and gagging and spitting part. Please.,Okay... then how 'bout we try out some French cuisine. Saturday? Night?,"[Not, the, hacking, and, gagging, and, spitting, part., Please.]","[Okay..., then, how, 'bout, we, try, out, some, French, cuisine., Saturday?, Night?]"
3,You're asking me out. That's so cute. What's your name again?,Forget it.,"[you are, asking, me, out., that is, so, cute., what is, your, name, again?]","[Forget, it.]"
4,"No, no, it's my fault -- we didn't have a proper introduction ---",Cameron.,"[No,, no,, it is, my, fault, --, we, did not, have, a, proper, introduction, ---]",[Cameron.]


In [60]:
data['question_tokenized'] = [' '.join(map(str, l)) for l in data['question_tokenized']]
data['answer_tokenized'] = [' '.join(map(str, l)) for l in data['answer_tokenized']]
data.head()

,question,answer,question_tokenized,answer_tokenized
0,Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again.,"Well, I thought we'd start with pronunciation, if that's okay with you.",Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again.,"Well, I thought we would start with pronunciation, if that is okay with you."
1,"Well, I thought we'd start with pronunciation, if that's okay with you.",Not the hacking and gagging and spitting part. Please.,"Well, I thought we would start with pronunciation, if that is okay with you.",Not the hacking and gagging and spitting part. Please.
2,Not the hacking and gagging and spitting part. Please.,Okay... then how 'bout we try out some French cuisine. Saturday? Night?,Not the hacking and gagging and spitting part. Please.,Okay... then how 'bout we try out some French cuisine. Saturday? Night?
3,You're asking me out. That's so cute. What's your name again?,Forget it.,you are asking me out. that is so cute. what is your name again?,Forget it.
4,"No, no, it's my fault -- we didn't have a proper introduction ---",Cameron.,"No, no, it is my fault -- we did not have a proper introduction ---",Cameron.


In [61]:
data['question_tokenized'] = data['question_tokenized'].apply(nltk.word_tokenize)
data['answer_tokenized'] = data['answer_tokenized'].apply(nltk.word_tokenize)
data.head()

,question,answer,question_tokenized,answer_tokenized
0,Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again.,"Well, I thought we'd start with pronunciation, if that's okay with you.","[Can, we, make, this, quick, ?, Roxanne, Korrine, and, Andrew, Barrett, are, having, an, incredibly, horrendous, public, break-, up, on, the, quad, ., Again, .]","[Well, ,, I, thought, we, would, start, with, pronunciation, ,, if, that, is, okay, with, you, .]"
1,"Well, I thought we'd start with pronunciation, if that's okay with you.",Not the hacking and gagging and spitting part. Please.,"[Well, ,, I, thought, we, would, start, with, pronunciation, ,, if, that, is, okay, with, you, .]","[Not, the, hacking, and, gagging, and, spitting, part, ., Please, .]"
2,Not the hacking and gagging and spitting part. Please.,Okay... then how 'bout we try out some French cuisine. Saturday? Night?,"[Not, the, hacking, and, gagging, and, spitting, part, ., Please, .]","[Okay, ..., then, how, 'bout, we, try, out, some, French, cuisine, ., Saturday, ?, Night, ?]"
3,You're asking me out. That's so cute. What's your name again?,Forget it.,"[you, are, asking, me, out, ., that, is, so, cute, ., what, is, your, name, again, ?]","[Forget, it, .]"
4,"No, no, it's my fault -- we didn't have a proper introduction ---",Cameron.,"[No, ,, no, ,, it, is, my, fault, --, we, did, not, have, a, proper, introduction, --, -]","[Cameron, .]"


In [62]:
data['question_tokenized'] = data['question_tokenized'].apply(lambda x: [word.lower() for word in x])
data['answer_tokenized'] = data['answer_tokenized'].apply(lambda x: [word.lower() for word in x])
data.head()

,question,answer,question_tokenized,answer_tokenized
0,Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again.,"Well, I thought we'd start with pronunciation, if that's okay with you.","[can, we, make, this, quick, ?, roxanne, korrine, and, andrew, barrett, are, having, an, incredibly, horrendous, public, break-, up, on, the, quad, ., again, .]","[well, ,, i, thought, we, would, start, with, pronunciation, ,, if, that, is, okay, with, you, .]"
1,"Well, I thought we'd start with pronunciation, if that's okay with you.",Not the hacking and gagging and spitting part. Please.,"[well, ,, i, thought, we, would, start, with, pronunciation, ,, if, that, is, okay, with, you, .]","[not, the, hacking, and, gagging, and, spitting, part, ., please, .]"
2,Not the hacking and gagging and spitting part. Please.,Okay... then how 'bout we try out some French cuisine. Saturday? Night?,"[not, the, hacking, and, gagging, and, spitting, part, ., please, .]","[okay, ..., then, how, 'bout, we, try, out, some, french, cuisine, ., saturday, ?, night, ?]"
3,You're asking me out. That's so cute. What's your name again?,Forget it.,"[you, are, asking, me, out, ., that, is, so, cute, ., what, is, your, name, again, ?]","[forget, it, .]"
4,"No, no, it's my fault -- we didn't have a proper introduction ---",Cameron.,"[no, ,, no, ,, it, is, my, fault, --, we, did, not, have, a, proper, introduction, --, -]","[cameron, .]"


In [63]:
punc = string.punctuation
data['question_tokenized'] = data['question_tokenized'].apply(lambda x: [word for word in x if word not in punc])
data.head()

,question,answer,question_tokenized,answer_tokenized
0,Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again.,"Well, I thought we'd start with pronunciation, if that's okay with you.","[can, we, make, this, quick, roxanne, korrine, and, andrew, barrett, are, having, an, incredibly, horrendous, public, break-, up, on, the, quad, again]","[well, ,, i, thought, we, would, start, with, pronunciation, ,, if, that, is, okay, with, you, .]"
1,"Well, I thought we'd start with pronunciation, if that's okay with you.",Not the hacking and gagging and spitting part. Please.,"[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]","[not, the, hacking, and, gagging, and, spitting, part, ., please, .]"
2,Not the hacking and gagging and spitting part. Please.,Okay... then how 'bout we try out some French cuisine. Saturday? Night?,"[not, the, hacking, and, gagging, and, spitting, part, please]","[okay, ..., then, how, 'bout, we, try, out, some, french, cuisine, ., saturday, ?, night, ?]"
3,You're asking me out. That's so cute. What's your name again?,Forget it.,"[you, are, asking, me, out, that, is, so, cute, what, is, your, name, again]","[forget, it, .]"
4,"No, no, it's my fault -- we didn't have a proper introduction ---",Cameron.,"[no, no, it, is, my, fault, --, we, did, not, have, a, proper, introduction, --]","[cameron, .]"


In [64]:
def clean(x):
  for i,word in enumerate(x):
    wordNew = re.sub('[\-_,.`"\']', '', word)
    if len(wordNew)<1:
      x.remove(word)
    elif word != wordNew:
      x[i] = wordNew
  return x

data['question_tokenized'].apply(lambda x: clean(x))
data['answer_tokenized'].apply(lambda x: clean(x))
data.head()

,question,answer,question_tokenized,answer_tokenized
0,Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again.,"Well, I thought we'd start with pronunciation, if that's okay with you.","[can, we, make, this, quick, roxanne, korrine, and, andrew, barrett, are, having, an, incredibly, horrendous, public, break, up, on, the, quad, again]","[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]"
1,"Well, I thought we'd start with pronunciation, if that's okay with you.",Not the hacking and gagging and spitting part. Please.,"[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]","[not, the, hacking, and, gagging, and, spitting, part, please]"
2,Not the hacking and gagging and spitting part. Please.,Okay... then how 'bout we try out some French cuisine. Saturday? Night?,"[not, the, hacking, and, gagging, and, spitting, part, please]","[okay, then, how, bout, we, try, out, some, french, cuisine, saturday, ?, night, ?]"
3,You're asking me out. That's so cute. What's your name again?,Forget it.,"[you, are, asking, me, out, that, is, so, cute, what, is, your, name, again]","[forget, it]"
4,"No, no, it's my fault -- we didn't have a proper introduction ---",Cameron.,"[no, no, it, is, my, fault, we, did, not, have, a, proper, introduction]",[cameron]


In [65]:
data.head(50)

,question,answer,question_tokenized,answer_tokenized
0,Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again.,"Well, I thought we'd start with pronunciation, if that's okay with you.","[can, we, make, this, quick, roxanne, korrine, and, andrew, barrett, are, having, an, incredibly, horrendous, public, break, up, on, the, quad, again]","[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]"
1,"Well, I thought we'd start with pronunciation, if that's okay with you.",Not the hacking and gagging and spitting part. Please.,"[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]","[not, the, hacking, and, gagging, and, spitting, part, please]"
2,Not the hacking and gagging and spitting part. Please.,Okay... then how 'bout we try out some French cuisine. Saturday? Night?,"[not, the, hacking, and, gagging, and, spitting, part, please]","[okay, then, how, bout, we, try, out, some, french, cuisine, saturday, ?, night, ?]"
3,You're asking me out. That's so cute. What's your name again?,Forget it.,"[you, are, asking, me, out, that, is, so, cute, what, is, your, name, again]","[forget, it]"
4,"No, no, it's my fault -- we didn't have a proper introduction ---",Cameron.,"[no, no, it, is, my, fault, we, did, not, have, a, proper, introduction]",[cameron]
5,Cameron.,"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser. My sister. I can't date until she does.",[cameron],"[the, thing, is, cameron, i, am, at, the, mercy, of, a, particularly, hideous, breed, of, loser, my, sister, i, can, not, date, until, she, does]"
6,"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser. My sister. I can't date until she does.",Seems like she could get a date easy enough...,"[the, thing, is, cameron, i, am, at, the, mercy, of, a, particularly, hideous, breed, of, loser, my, sister, i, can, not, date, until, she, does]","[seems, like, she, could, get, a, date, easy, enough]"
7,Why?,"Unsolved mystery. She used to be really popular when she started high school, then it was just like she got sick of it or something.",[why],"[unsolved, mystery, she, used, to, be, really, popular, when, she, started, high, school, then, it, was, just, like, she, got, sick, of, it, or, something]"
8,"Unsolved mystery. She used to be really popular when she started high school, then it was just like she got sick of it or something.",That's a shame.,"[unsolved, mystery, she, used, to, be, really, popular, when, she, started, high, school, then, it, was, just, like, she, got, sick, of, it, or, something]","[that, is, a, shame]"
9,"Gosh, if only we could find Kat a boyfriend...",Let me see what I can do.,"[gosh, if, only, we, could, find, kat, a, boyfriend]","[let, me, see, what, i, can, do]"


In [66]:
word_count = {}
for question in data['question_tokenized']:
    for word in question:
        if word not in word_count:
            word_count[word] = 1
        else:
            word_count[word] += 1
for answer in data['answer_tokenized']:
    for word in answer:
        if word not in word_count:
            word_count[word] = 1
        else:
            word_count[word] += 1
print(len(word_count))
print(word_count)

6586
{'can': 627, 'we': 930, 'make': 140, 'this': 757, 'quick': 2, 'roxanne': 1, 'korrine': 1, 'and': 1287, 'andrew': 1, 'barrett': 1, 'are': 1382, 'having': 23, 'an': 182, 'incredibly': 6, 'horrendous': 1, 'public': 6, 'break': 15, 'up': 369, 'on': 594, 'the': 2991, 'quad': 1, 'again': 57, 'well': 316, 'i': 4459, 'thought': 82, 'would': 477, 'start': 51, 'with': 570, 'pronunciation': 2, 'if': 404, 'that': 1392, 'is': 2715, 'okay': 165, 'you': 4897, 'not': 2533, 'hacking': 2, 'gagging': 2, 'spitting': 2, 'part': 33, 'please': 92, 'asking': 25, 'me': 1181, 'out': 416, 'so': 392, 'cute': 3, 'what': 1142, 'your': 765, 'name': 51, 'no': 584, 'it': 2146, 'my': 731, 'fault': 16, 'did': 482, 'have': 1038, 'a': 2310, 'proper': 5, 'introduction': 1, 'cameron': 11, 'thing': 135, 'am': 895, 'at': 300, 'mercy': 2, 'of': 1206, 'particularly': 5, 'hideous': 2, 'breed': 2, 'loser': 3, 'sister': 29, 'date': 22, 'until': 42, 'she': 404, 'does': 154, 'why': 283, 'unsolved': 2, 'mystery': 2, 'used': 38, 

In [67]:
threshold= 2
words_indexed = {}
word_number = 4

tokens = ['<PAD>', '<OUT>', '<SOS>', '<EOS>']
for token in tokens:
    words_indexed[token] = len(words_indexed)

for word, count in word_count.items():
    if count >= threshold:
        words_indexed[word] = word_number
        word_number += 1

sizeOfVocab = len(words_indexed)
print(sizeOfVocab)
print(words_indexed)


4380
{'<PAD>': 0, '<OUT>': 1, '<SOS>': 2, '<EOS>': 3, 'can': 4, 'we': 5, 'make': 6, 'this': 7, 'quick': 8, 'and': 9, 'are': 10, 'having': 11, 'an': 12, 'incredibly': 13, 'public': 14, 'break': 15, 'up': 16, 'on': 17, 'the': 18, 'again': 19, 'well': 20, 'i': 21, 'thought': 22, 'would': 23, 'start': 24, 'with': 25, 'pronunciation': 26, 'if': 27, 'that': 28, 'is': 29, 'okay': 30, 'you': 31, 'not': 32, 'hacking': 33, 'gagging': 34, 'spitting': 35, 'part': 36, 'please': 37, 'asking': 38, 'me': 39, 'out': 40, 'so': 41, 'cute': 42, 'what': 43, 'your': 44, 'name': 45, 'no': 46, 'it': 47, 'my': 48, 'fault': 49, 'did': 50, 'have': 51, 'a': 52, 'proper': 53, 'cameron': 54, 'thing': 55, 'am': 56, 'at': 57, 'mercy': 58, 'of': 59, 'particularly': 60, 'hideous': 61, 'breed': 62, 'loser': 63, 'sister': 64, 'date': 65, 'until': 66, 'she': 67, 'does': 68, 'why': 69, 'unsolved': 70, 'mystery': 71, 'used': 72, 'to': 73, 'be': 74, 'really': 75, 'popular': 76, 'when': 77, 'started': 78, 'high': 79, 'school'

In [68]:
for question in data['question_tokenized']:
    question.append('<EOS>')
    question.insert(0,'<SOS>')

for answer in data['answer_tokenized']:
    answer.append('<EOS>')
    answer.insert(0,'<SOS>')

In [69]:
print(data['question_tokenized'])

0        [<SOS>, can, we, make, this, quick, roxanne, korrine, and, andrew, barrett, are, having, an, incredibly, horrendous, public, break, up, on, the, quad, again, <EOS>]
1                                                                      [<SOS>, well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you, <EOS>]
2                                                                                                [<SOS>, not, the, hacking, and, gagging, and, spitting, part, please, <EOS>]
3                                                                                  [<SOS>, you, are, asking, me, out, that, is, so, cute, what, is, your, name, again, <EOS>]
4                                                                                      [<SOS>, no, no, it, is, my, fault, we, did, not, have, a, proper, introduction, <EOS>]
                                                                                         ...                                      

In [70]:
def convertToInt(inputSen):
    global words_indexed
    sen_into_int = []
    for word in inputSen:
        if word not in words_indexed:
            sen_into_int.append(words_indexed['<OUT>'])
        else:
            sen_into_int.append(words_indexed[word])
    return sen_into_int

data['question_tokenized_int'] = data['question_tokenized'].apply(lambda x: convertToInt(x))
data['answer_tokenized_int'] = data['answer_tokenized'].apply(lambda x: convertToInt(x))

In [71]:
data.head(50)

,question,answer,question_tokenized,answer_tokenized,question_tokenized_int,answer_tokenized_int
0,Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again.,"Well, I thought we'd start with pronunciation, if that's okay with you.","[<SOS>, can, we, make, this, quick, roxanne, korrine, and, andrew, barrett, are, having, an, incredibly, horrendous, public, break, up, on, the, quad, again, <EOS>]","[<SOS>, well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you, <EOS>]","[2, 4, 5, 6, 7, 8, 1, 1, 9, 1, 1, 10, 11, 12, 13, 1, 14, 15, 16, 17, 18, 1, 19, 3]","[2, 20, 21, 22, 5, 23, 24, 25, 26, 27, 28, 29, 30, 25, 31, 3]"
1,"Well, I thought we'd start with pronunciation, if that's okay with you.",Not the hacking and gagging and spitting part. Please.,"[<SOS>, well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you, <EOS>]","[<SOS>, not, the, hacking, and, gagging, and, spitting, part, please, <EOS>]","[2, 20, 21, 22, 5, 23, 24, 25, 26, 27, 28, 29, 30, 25, 31, 3]","[2, 32, 18, 33, 9, 34, 9, 35, 36, 37, 3]"
2,Not the hacking and gagging and spitting part. Please.,Okay... then how 'bout we try out some French cuisine. Saturday? Night?,"[<SOS>, not, the, hacking, and, gagging, and, spitting, part, please, <EOS>]","[<SOS>, okay, then, how, bout, we, try, out, some, french, cuisine, saturday, ?, night, ?, <EOS>]","[2, 32, 18, 33, 9, 34, 9, 35, 36, 37, 3]","[2, 30, 81, 104, 1570, 5, 887, 40, 503, 399, 2827, 218, 4233, 250, 4233, 3]"
3,You're asking me out. That's so cute. What's your name again?,Forget it.,"[<SOS>, you, are, asking, me, out, that, is, so, cute, what, is, your, name, again, <EOS>]","[<SOS>, forget, it, <EOS>]","[2, 31, 10, 38, 39, 40, 28, 29, 41, 42, 43, 29, 44, 45, 19, 3]","[2, 253, 47, 3]"
4,"No, no, it's my fault -- we didn't have a proper introduction ---",Cameron.,"[<SOS>, no, no, it, is, my, fault, we, did, not, have, a, proper, introduction, <EOS>]","[<SOS>, cameron, <EOS>]","[2, 46, 46, 47, 29, 48, 49, 5, 50, 32, 51, 52, 53, 1, 3]","[2, 54, 3]"
5,Cameron.,"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser. My sister. I can't date until she does.","[<SOS>, cameron, <EOS>]","[<SOS>, the, thing, is, cameron, i, am, at, the, mercy, of, a, particularly, hideous, breed, of, loser, my, sister, i, can, not, date, until, she, does, <EOS>]","[2, 54, 3]","[2, 18, 55, 29, 54, 21, 56, 57, 18, 58, 59, 52, 60, 61, 62, 59, 63, 48, 64, 21, 4, 32, 65, 66, 67, 68, 3]"
6,"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser. My sister. I can't date until she does.",Seems like she could get a date easy enough...,"[<SOS>, the, thing, is, cameron, i, am, at, the, mercy, of, a, particularly, hideous, breed, of, loser, my, sister, i, can, not, date, until, she, does, <EOS>]","[<SOS>, seems, like, she, could, get, a, date, easy, enough, <EOS>]","[2, 18, 55, 29, 54, 21, 56, 57, 18, 58, 59, 52, 60, 61, 62, 59, 63, 48, 64, 21, 4, 32, 65, 66, 67, 68, 3]","[2, 410, 84, 67, 90, 135, 52, 65, 892, 297, 3]"
7,Why?,"Unsolved mystery. She used to be really popular when she started high school, then it was just like she got sick of it or something.","[<SOS>, why, <EOS>]","[<SOS>, unsolved, mystery, she, used, to, be, really, popular, when, she, started, high, school, then, it, was, just, like, she, got, sick, of, it, or, something, <EOS>]","[2, 69, 3]","[2, 70, 71, 67, 72, 73, 74, 75, 76, 77, 67, 78, 79, 80, 81, 47, 82, 83, 84, 67, 85, 86, 59, 47, 87, 88, 3]"
8,"Unsolved mystery. She used to be really popular when she started high school, then it was just like she got sick of it or something.",That's a shame.,"[<SOS>, unsolved, mystery, she, used, to, be, really, popular, when, she, started, high, school, then, it, was, just, like, she, got, sick, of, it, or, something, <EOS>]","[<SOS>, that, is, a, shame, <EOS>]","[2, 70, 71, 67, 72, 73, 74, 75, 76, 77, 67, 78

In [72]:
def converToTensor(question,answer):
    question = question.to_tensor()
    answer = answer.to_tensor()
    return question,answer

In [73]:
#data[data.columns[0:2]].values
#testingData=tf.ragged.constant(data[data.columns[4:6]].values)
#data

In [74]:
#print(testingData[0][1])

In [75]:
#testingOutput=[([2,5,3],[2,4,1]),([2,8,3],[2,4,3])]
#hh=tf.convert_to_tensor(testingOutput)
#print(type(hh[0]))
#testingTensor = tf.ragged.constant(testingOutput)
#print(type(testingTensor))
#print(testingTensor[0])
#datasetNew = tf.data.Dataset.from_tensor_slices(testingTensor)
#for x in testingData:
#  print(x)
#  break
#print(testingData.to_tensor())

In [76]:
#datasetNew = tf.data.Dataset.from_tensor_slices(testingData)
#tensorData = testingData.to_tensor()
#tensorData
#datasetNew = tf.data.Dataset.from_tensor_slices(tensorData)
#tensorData=tf.expand_dims(tensorData,axis=1)
#tensorData
#for x in tensorData:
  #print(tf.reshape(x, [,]))
  #print(tf.expand_dims(x,axis=0))
 # print(x)
 # break
#datasetNew = tf.reshape(testingData, [,])
#print(testingData.get_shape())

In [77]:
#print(type(data['question_tokenized_int'].values))
listOfQuestionsNpArray=data['question_tokenized_int'].values
listOfAnswersNpArray=data['answer_tokenized_int'].values
tensorQuestions=tf.ragged.constant(listOfQuestionsNpArray,dtype=tf.int64)
tensorAnswers=tf.ragged.constant(listOfAnswersNpArray,dtype=tf.int64)
datasetNew = tf.data.Dataset.from_tensor_slices((tensorQuestions,tensorAnswers))


In [78]:
def convertSentenceToTensor(sentence):
  sentence = [contractions.fix(word) for word in str(sentence).split()]
  #print(sentence)
  sentence = ' '.join(map(str, sentence))
  #print(sentence)
  sentence = str(sentence).split()
  #print(sentence)
  sentence = [word.lower() for word in sentence]
  #print(sentence)
  sentence = [word for word in sentence if word not in punc]
  #print(sentence)
  sentence = clean(sentence)
  #print(sentence)
  sentence.append('<EOS>')
  sentence.insert(0,'<SOS>')
  #print(sentence)
  sentence = convertToInt(sentence)
  #print(sentence)
  sentence = tf.convert_to_tensor(sentence, dtype=tf.int64)
  #print(sentence)
  sentence = tf.expand_dims(sentence, 0)
  print(sentence)
  return sentence

In [79]:
convertSentenceToTensor("Hi, I'm robot and my name is \"John\" ! ? $")

tf.Tensor([[   2  158   21   56    1    9   48   45   29 2640    3]], shape=(1, 11), dtype=int64)


<tf.Tensor: shape=(1, 11), dtype=int64, numpy=array([[   2,  158,   21,   56,    1,    9,   48,   45,   29, 2640,    3]])>

In [80]:
#for x in datasetNew:
#  print(x)
#  break

In [81]:

#datasetNew = tf.data.Dataset.from_tensor_slices(hh)
#print(type(datasetNew))
#for x in datasetNew:
#  print(x)
#  break

In [82]:
BUFFER_SIZE = 2500
BATCH_SIZE = 32

In [83]:
def make_batches(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(converToTensor, num_parallel_calls=tf.data.AUTOTUNE)
      .prefetch(tf.data.AUTOTUNE))


train_batches = make_batches(datasetNew)

In [84]:
for x in train_batches:
  print("outB:")
  print(x)
  break

outB:
(<tf.Tensor: shape=(32, 69), dtype=int64, numpy=
array([[  2, 104,   3, ...,   0,   0,   0],
       [  2,  20,  31, ...,   0,   0,   0],
       [  2, 258,  28, ...,   0,   0,   0],
       ...,
       [  2, 131,  27, ...,   0,   0,   0],
       [  2,  31, 261, ...,   0,   0,   0],
       [  2, 229, 423, ...,   0,   0,   0]])>, <tf.Tensor: shape=(32, 32), dtype=int64, numpy=
array([[  2,  83, 248, ...,   0,   0,   0],
       [  2,  43, 232, ...,   0,   0,   0],
       [  2, 188,  29, ...,   0,   0,   0],
       ...,
       [  2,   7, 347, ...,   0,   0,   0],
       [  2, 703,  18, ...,   0,   0,   0],
       [  2,  21,  22, ...,   0,   0,   0]])>)


In [85]:
#print(type(train_batches))
#for x in train_batches:
#  print("outPT:")
#  print(x[0][0])
#  print("outEN:")
#  print(x[1][0])
#  print("out:")
#  print(fromIntToWords(tf.expand_dims(x[0][0],axis=0)))
#  print("out2:")
#  print(fromIntToWords(tf.expand_dims(x[1][0],axis=0)))
#  break

Transformer architecture source: https://www.tensorflow.org/text/tutorials/transformer

In [86]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)


In [87]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

In [88]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights
def print_out(q, k, v):
  temp_out, temp_attn = scaled_dot_product_attention(
      q, k, v, None)
  print('Attention weights are:')
  print(temp_attn)
  print('Output is:')
  print(temp_out)
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

In [89]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [90]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

    return out3, attn_weights_block1, attn_weights_block2

In [91]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding,
                                            self.d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]

    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x  # (batch_size, input_seq_len, d_model)
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [92]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, answer_input, question_target, rate=0.1):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size, answer_input, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                           target_vocab_size, question_target, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inp, tar, training, enc_padding_mask,
           look_ahead_mask, dec_padding_mask):

    enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)

    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

    return final_output, attention_weights

In [93]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [94]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [95]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [96]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [97]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


def accuracy_function(real, pred):
  accuracies = tf.equal(real, tf.argmax(pred, axis=2))

  mask = tf.math.logical_not(tf.math.equal(real, 0))
  accuracies = tf.math.logical_and(mask, accuracies)

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

In [98]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

In [99]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=sizeOfVocab + 1,
    target_vocab_size=sizeOfVocab + 1,
    answer_input=1000,
    question_target=1000,
    rate=dropout_rate)

In [100]:
def create_masks(inp, tar):
  # Encoder padding mask
  enc_padding_mask = create_padding_mask(inp)

  # Used in the 2nd attention block in the decoder.
  # This padding mask is used to mask the encoder outputs.
  dec_padding_mask = create_padding_mask(inp)

  # Used in the 1st attention block in the decoder.
  # It is used to pad and mask future tokens in the input received by
  # the decoder.
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  dec_target_padding_mask = create_padding_mask(tar)
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

  return enc_padding_mask, combined_mask, dec_padding_mask

In [101]:
#checkpoint_path = "./checkpoints2/train"

checkpoint_path = "/content/drive/MyDrive/checkpoints1/train"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [102]:
EPOCHS = 50

In [103]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),#tf.int64
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]


@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]

  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

  with tf.GradientTape() as tape:
    predictions, _ = transformer(inp, tar_inp,
                                 True,
                                 enc_padding_mask,
                                 combined_mask,
                                 dec_padding_mask)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

  train_loss(loss)
  train_accuracy(accuracy_function(tar_real, predictions))

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  train_loss.reset_states()
  train_accuracy.reset_states()

  for (batch, (inp, tar)) in enumerate(train_batches):
    train_step(inp, tar)

    if batch % 50 == 0:
      print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print(f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

  print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')

Epoch 1 Batch 0 Loss 8.6458 Accuracy 0.0000
Epoch 1 Batch 50 Loss 8.5779 Accuracy 0.0140
Epoch 1 Batch 100 Loss 8.4583 Accuracy 0.0470


KeyboardInterrupt: ignored

In [ ]:
#ckpt_save_path = ckpt_manager.save()

In [104]:
def returnTokens():
  start = tf.convert_to_tensor(2, dtype=tf.int64)
  end = tf.convert_to_tensor(3, dtype=tf.int64)
  return start, end

In [105]:
print(returnTokens()[0])

tf.Tensor(2, shape=(), dtype=int64)


In [106]:
inv_words_indexed = {value:key for key, value in words_indexed.items()}
def fromIntToWords(output):
  sentence = []
  for number in output[0]:
    #print(inv_words_indexed[number])
    #with tf.Session() as sess:  print(number.eval())
    word = inv_words_indexed[number.numpy()]
    if number.numpy() not in [1,2,3]:
      sentence.append(word)
  sentence = " ".join(sentence)
  return sentence

In [107]:
fromIntToWords(convertSentenceToTensor("Hi, this is test."))

tf.Tensor([[   2  158    7   29 1435    3]], shape=(1, 6), dtype=int64)


'hi this is test'

In [108]:
def evaluate(sentence, max_length=40):

  sentence = convertSentenceToTensor(sentence)
  encoder_input = sentence
  start, end = returnTokens()
  output = tf.convert_to_tensor([start])
  output = tf.expand_dims(output, 0)

  for i in range(max_length):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_input, output)

    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights = transformer(encoder_input,
                                                 output,
                                                 False,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask)

    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.argmax(predictions, axis=-1)
    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)
    # return the result if the predicted_id is equal to the end token
    if predicted_id == end:
      break

  # output.shape (1, tokens)
  text = fromIntToWords(output)  # shape: ()
  return text

In [109]:
def print_translation(sentence, tokens):
  print(f'{"Input:":15s}: {sentence}')
  print(f'{"Prediction":15s}: {tokens}')

In [114]:
sentence = "Hi"

translated_text = evaluate(sentence)
print_translation(sentence, translated_text)

tf.Tensor([[  2 158   3]], shape=(1, 3), dtype=int64)
Input:         : Hi
Prediction     : in your dreams ! see you tonight !


In [124]:
sentence = "I love to dream, who are you"

translated_text = evaluate(sentence)
print_translation(sentence, translated_text)

tf.Tensor([[   2   21  408   73 1349  190   10   31    3]], shape=(1, 9), dtype=int64)
Input:         : I love to dream, who are you
Prediction     : we are with the police department this is sergeant sykes and i am
